In [1]:
import dwave
import numpy as np
import openml
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import plotly.express as px
from plotly.subplots import make_subplots
import dimod
from dwave.system import LeapHybridCQMSampler

# dataset = openml.datasets.get_dataset(312)

clf = RandomForestClassifier()

In [2]:
df = pd.read_csv("Dataset-stock-JNJ.csv")

In [3]:
df.head()

,Date,Close,High,Low,SMA5,SMA10,SMA20,SMA30,RSI,%K,...,Aroon Up 10,Aroon Down 10,Aroon Up 15,Aroon Down 15,Aroon Up 20,Aroon Down 20,Aroon Up 30,Aroon Down 30,Price Change,Class
0,2021-02-09,166.270004,166.589996,164.970001,163.626001,164.226003,163.306501,161.378000,57.019124,47.585251,...,90.0,50.0,73.333333,33.333333,55.0,95.0,36.666667,100.000000,-0.539993,1
1,2021-02-10,166.809998,167.940002,165.699997,164.888000,164.119002,163.740501,161.832000,59.202905,51.420434,...,100.0,60.0,80.000000,40.000000,60.0,100.0,40.000000,100.000000,0.750000,0
2,2021-02-11,166.059998,167.380005,165.600006,165.701999,163.809001,164.149001,162.229333,54.730486,46.093725,...,20.0,70.0,86.666667,46.666667,65.0,95.0,43.333333,100.000000,-0.520004,1
3,2021-02-12,166.580002,166.850006,164.000000,166.128000,164.154001,164.445501,162.580333,51.218534,49.786940,...,30.0,80.0,93.333333,53.333333,70.0,100.0,46.666667,96.666667,1.509995,0
4,2021-02-16,165.070007,167.050003,164.639999,166.158002,164.390001,164.684001,162.836667,37.494242,48.458189,...,40.0,90.0,100.000000,60.000000,75.0,45.0,50.000000,100.000000,-0.589996,1


In [4]:
df.drop('Price Change',axis=1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           473 non-null    object 
 1   Close          473 non-null    float64
 2   High           473 non-null    float64
 3   Low            473 non-null    float64
 4   SMA5           473 non-null    float64
 5   SMA10          473 non-null    float64
 6   SMA20          473 non-null    float64
 7   SMA30          473 non-null    float64
 8   RSI            473 non-null    float64
 9   %K             473 non-null    float64
 10  %D             473 non-null    float64
 11  ATR            473 non-null    float64
 12  ADX            473 non-null    float64
 13  EMA5           473 non-null    float64
 14  EMA10          473 non-null    float64
 15  EMA20          473 non-null    float64
 16  EMA30          473 non-null    float64
 17  Aroon Up 5     473 non-null    float64
 18  Aroon Down

In [6]:
# from sklearn.datasets import load_breast_cancer
from dwave.plugins.sklearn import SelectFromQuadraticModel

X = df.drop(['Class','Date'], axis=1)
y = df['Class']


X.head()

,Close,High,Low,SMA5,SMA10,SMA20,SMA30,RSI,%K,%D,...,Aroon Up 5,Aroon Down 5,Aroon Up 10,Aroon Down 10,Aroon Up 15,Aroon Down 15,Aroon Up 20,Aroon Down 20,Aroon Up 30,Aroon Down 30
0,166.270004,166.589996,164.970001,163.626001,164.226003,163.306501,161.378000,57.019124,47.585251,40.080477,...,20.0,100.0,90.0,50.0,73.333333,33.333333,55.0,95.0,36.666667,100.000000
1,166.809998,167.940002,165.699997,164.888000,164.119002,163.740501,161.832000,59.202905,51.420434,45.667605,...,20.0,100.0,100.0,60.0,80.000000,40.000000,60.0,100.0,40.000000,100.000000
2,166.059998,167.380005,165.600006,165.701999,163.809001,164.149001,162.229333,54.730486,46.093725,48.366470,...,40.0,100.0,20.0,70.0,86.666667,46.666667,65.0,95.0,43.333333,100.000000
3,166.580002,166.850006,164.000000,166.128000,164.154001,164.445501,162.580333,51.218534,49.786940,49.100366,...,60.0,100.0,30.0,80.0,93.333333,53.333333,70.0,100.0,46.666667,96.666667
4,165.070007,167.050003,164.639999,166.158002,164.390001,164.684001,162.836667,37.494242,48.458189,48.112951,...,80.0,20.0,40.0,90.0,100.000000,60.000000,75.0,45.0,50.000000,100.000000


In [7]:
X_new = SelectFromQuadraticModel(num_features=5).fit_transform(X, y)


/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [8]:
print(X_new.shape)

X_new

(473, 5)


array([[163.30650101,   3.41642761,  22.66245525,  20.        ,
         95.        ],
       [163.74050064,   3.40357099,  21.96934638,  20.        ,
        100.        ],
       [164.14900055,   3.2857132 ,  20.28787737,  40.        ,
         95.        ],
       ...,
       [177.31100082,   2.23499952,  14.98611376, 100.        ,
         10.        ],
       [177.35400009,   2.25928497,  16.29650876,  20.        ,
         15.        ],
       [177.35500031,   2.28357043,  14.64997528,  40.        ,
         20.        ]])

In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score

In [10]:
def train_test_models(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create a list to store the results
    results = []
    
    # Models
    models = {
        'SVM': SVC(),
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'KNN': KNeighborsClassifier(),
        'Logistic Regression': LogisticRegression(max_iter=1500),
        'Naive Bayes': GaussianNB(),
        'Gradient Boosting': GradientBoostingClassifier(),
        'XGBoost': xgb.XGBClassifier()
    }
    
    for model_name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Calculate accuracy and F1 score
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Store the results as a dictionary
        results.append({'Model': model_name, 'Accuracy': accuracy, 'F1 Score': f1})
    
    # Create a DataFrame from the results list
    results_df = pd.DataFrame(results)
    
    return results_df

In [11]:

# Train and test the models
results = train_test_models(X_new, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.505263,0.560748
1,Decision Tree,0.536842,0.488372
2,Random Forest,0.526316,0.563107
3,KNN,0.368421,0.400000
4,Logistic Regression,0.557895,0.562500
5,Naive Bayes,0.494737,0.500000
6,Gradient Boosting,0.463158,0.451613
7,XGBoost,0.568421,0.577320


In [12]:
X_new3 = SelectFromQuadraticModel(num_features=3).fit_transform(X, y)


/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [13]:
# Train and test the models
results = train_test_models(X_new3, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.442105,0.361446
1,Decision Tree,0.515789,0.540000
2,Random Forest,0.463158,0.463158
3,KNN,0.400000,0.412371
4,Logistic Regression,0.505263,0.471910
5,Naive Bayes,0.452632,0.422222
6,Gradient Boosting,0.452632,0.480000
7,XGBoost,0.494737,0.489362


In [14]:
X_new8 = SelectFromQuadraticModel(num_features=8).fit_transform(X, y)

/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [15]:
# Train and test the models
results = train_test_models(X_new8, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.505263,0.534653
1,Decision Tree,0.536842,0.521739
2,Random Forest,0.536842,0.576923
3,KNN,0.421053,0.476190
4,Logistic Regression,0.547368,0.574257
5,Naive Bayes,0.557895,0.596154
6,Gradient Boosting,0.494737,0.520000
7,XGBoost,0.536842,0.560000


In [16]:
num_cols = X_new.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa5 = pd.DataFrame(data=X_new, columns=col_names)

# Save the DataFrame to a CSV file
qa5.to_csv('QA-5-features-JNJ.csv', index=False)

In [17]:
num_cols = X_new3.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa3 = pd.DataFrame(data=X_new3, columns=col_names)

# Save the DataFrame to a CSV file
qa3.to_csv('QA-3-features-JNJ.csv', index=False)

In [18]:
num_cols = X_new8.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa8 = pd.DataFrame(data=X_new8, columns=col_names)

# Save the DataFrame to a CSV file
qa8.to_csv('QA-8-features-JNJ.csv', index=False)